In [3]:
import sys
import os
import dlib
import glob
from skimage import io

# Note: Made extensive use of https://github.com/davisking/dlib/blob/master/python_examples/face_recognition.py

In [7]:
predictor_path = "/face_rec/shape_predictor_5_face_landmarks.dat"
face_rec_model_path = "/dlib_face_recognition_resnet_model_v1.dat"
faces_folder_path = "/face_rec/faces"

# Load all the models we need: a detector to find the faces, a shape predictor
# to find face landmarks so we can precisely localize the face, and finally the
# face recognition model.
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

In [39]:
def process_image(img):
    """ Given image
    Returns an list of all faces, their location and their 128D vector
    If two faces have, between them, an Euclidian distance of less 
    than 0.6 they are likely to be the same person.
    """
    
    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time. This
    # will make everything bigger and allow us to detect more faces.
    dets = detector(img, 1)
    faces = []
    print("Number of faces detected: {}".format(len(dets)))
    
    # Process each face we found.
    for k, d in enumerate(dets):
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(k, d.left(), d.top(), d.right(), d.bottom()))
        # Get the landmarks/parts for the face in box d.
        shape = sp(img, d)
        
        # Compute the 128D vector that describes the face in img identified by
        # shape.
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        # face_descriptor = facerec.compute_face_descriptor(img, shape, 100) # Higher accuracy (99.13 to 99.38, 100 times slower)

        faces.append({
            'box': d,
            'vector': face_descriptor
        })
    return faces

In [40]:
# Now process all the images
images = []
for f in glob.glob(os.path.join(faces_folder_path, "*.jpg")):
    print("Processing file: {}".format(f))
    img = io.imread(f)
    
    images.append({
        'filename': f,
        'image': img,
        'result': process_image(img)
    })

Processing file: /Users/h.hendriksen/Desktop/face_rec/faces/bald_guys.jpg
Number of faces detected: 24
Detection 0: Left: 1457 Top: 1343 Right: 1586 Bottom: 1472
Detection 1: Left: 1067 Top: 1295 Right: 1253 Bottom: 1481
Detection 2: Left: 56 Top: 77 Right: 242 Bottom: 263
Detection 3: Left: 1047 Top: 758 Right: 1232 Bottom: 944
Detection 4: Left: 551 Top: 428 Right: 737 Bottom: 614
Detection 5: Left: 1823 Top: 438 Right: 1931 Bottom: 546
Detection 6: Left: 35 Top: 1274 Right: 221 Bottom: 1460
Detection 7: Left: 786 Top: 821 Right: 941 Bottom: 976
Detection 8: Left: 64 Top: 700 Right: 218 Bottom: 855
Detection 9: Left: 64 Top: 391 Right: 218 Bottom: 546
Detection 10: Left: 322 Top: 442 Right: 476 Bottom: 597
Detection 11: Left: 1658 Top: 703 Right: 1748 Bottom: 793
Detection 12: Left: 340 Top: 1232 Right: 563 Bottom: 1455
Detection 13: Left: 861 Top: 407 Right: 1047 Bottom: 593
Detection 14: Left: 67 Top: 999 Right: 196 Bottom: 1128
Detection 15: Left: 390 Top: 811 Right: 613 Bottom: 1

In [41]:
images

[{'filename': '/Users/h.hendriksen/Desktop/face_rec/faces/bald_guys.jpg',
  'image': array([[[ 97,  93,  81],
          [100,  96,  84],
          [102,  98,  86],
          ...,
          [182, 175, 169],
          [182, 173, 166],
          [180, 171, 164]],
  
         [[ 96,  92,  80],
          [ 98,  94,  82],
          [100,  96,  84],
          ...,
          [182, 175, 169],
          [182, 173, 166],
          [180, 171, 164]],
  
         [[ 99,  95,  83],
          [ 99,  95,  83],
          [ 99,  95,  83],
          ...,
          [182, 175, 169],
          [182, 173, 166],
          [180, 171, 164]],
  
         ...,
  
         [[  9,  10,  12],
          [ 10,  11,  13],
          [ 10,  11,  13],
          ...,
          [ 92,  31, 134],
          [ 92,  31, 135],
          [ 92,  31, 135]],
  
         [[  9,   9,  11],
          [ 11,  11,  13],
          [ 11,  11,  13],
          ...,
          [ 83,  40, 112],
          [ 82,  40, 112],
          [ 82,  40, 112]]